In [10]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
client = OpenAI(base_url="https://openai.vocareum.com/v1")

In [12]:
class Memory:
  def __init__(self):
    self.messages: List[Dict[str, str]] = []

  def add_message(self, role: Literal["user", "system", "assistant"], content: str):
    self.messages.append({"role": role, "content": content})
  
  def get_messages(self) -> List[Dict[str, str]]:
    return self.messages
  
  def last_message(self) -> Dict[str, str]:
    if self.messages:
      return self.messages[-1]


In [13]:
SELF_CRITIQUE_PROMPT = """
Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise.
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
"""

In [14]:
class Agent:
  """A self-reflection AI Agent"""
  
  def __init__(
    self,
    name: str = "Agent",
    role: str = "Personal Assistant",
    instructions: str = "Help users with any question.",
    model: str = "gpt-4o-mini",
    temperature: float = 0.0,
    ):
    self.name = name
    self.role = role
    self.instructions = instructions
    self.model = model
    self.temperature = temperature
    
    self.client = client
    self.memory = Memory()
    self.memory.add_message(
      role="system",
      content=f"You're an AI Agent, your rolse is {self.role}, "
              f"and you need to {self.instructions}"
    )
    
    self.critique_prompt = SELF_CRITIQUE_PROMPT

  
  def invoke(self, 
             user_message: str, 
             self_reflection: bool = False, 
             max_iter: int = 1,
             verbose: bool = False) -> str:
    
    self.memory.add_message(
      role="user",
      content=user_message
    )
    
    if verbose:
      self._log_last_message()
    
    max_iter = max_iter if max_iter >= 1 else 1
    max_iter = max_iter if max_iter <= 3 else 3
    max_iter = max_iter if self_reflection else 0.5
    loops = 2 * max_iter
    
    for i in range(loops):
      ai_message = self._get_completion(
        messages = self.memory.get_messages()
      )
      
      self.memory.add_message(
        role = "assistant",
        content = ai_message.content,
      )
      
      if verbose:
        self._log_last_message()
      
      if i < loops -1:
        self.memory.add_message(
          role = "user",
          content = self.critique_prompt
        )
        
        if verbose:
          self._log_last_message()
        
        ai_message = self._get_completion(
          messages = self.memory.get_messages()
        )

  def _get_completion(self, messages: List[Dict]) -> ChatCompletionMessage:
    response = self.client.chat.completions.create(
      model=self.model,
      temperature=self.temperature,
      messages=messages
    )
    
    return response.choices[0].message
  
  def _log_last_message(self):
    print(f"### {self.memory.last_message()['role']} message ### \n".upper())
    print(f"### {self.memory.last_message()['content']} \n")
    print("\n__________________________________________________________________")


In [15]:
agent = Agent()
agent.invoke(
  user_message="Pick only one. Who is the best character in Game of Thrones?",
  self_reflection=True,
  verbose=True
)

### USER MESSAGE ### 

### Pick only one. Who is the best character in Game of Thrones? 


__________________________________________________________________
### ASSISTANT MESSAGE ### 

### Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and moral complexity. He navigates the treacherous political landscape with a unique perspective and often serves as a voice of reason amidst chaos. Who is your favorite character? 


__________________________________________________________________
### USER MESSAGE ### 

### 
Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise.
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
 


_________________________________

In [16]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI Agent, your rolse is Personal Assistant, and you need to Help users with any question."},
 {'role': 'user',
  'content': 'Pick only one. Who is the best character in Game of Thrones?'},
 {'role': 'assistant',
  'content': 'Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and moral complexity. He navigates the treacherous political landscape with a unique perspective and often serves as a voice of reason amidst chaos. Who is your favorite character?'},
 {'role': 'user',
  'content': '\nReflect on your previous response...\nIdentify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise.\nProvide a revised response if necessary in a Json Output structure:\n{\n    "original_response": "",\n    "revisions_needed": "",\n    "updated_response": ""\n}\n'},
 {'rol

In [17]:
json.loads(agent.memory.last_message()["content"])["updated_response"]

"Many fans consider Tyrion Lannister the best character in 'Game of Thrones' due to his wit, intelligence, and moral complexity. He effectively navigates the political landscape and often serves as a voice of reason. Other notable characters include Jon Snow and Daenerys Targaryen, each with their own compelling arcs."